## Get to know the data

In [1]:
#import the data shit
from IPython.display import display as d
from pandas.io.formats.info import DataFrameInfoPrinter
import os
import pandas as pd
import numpy as np
import pandas as pd
#import the decision tree shit
#getting to the data directory
data_dir = 'devided_dataset_v2'
categories = ['CDs_and_Vinyl', 'Grocery_and_Gourmet_Food', 'Toys_and_Games']

In [2]:
#getting the products and the reviews for the shit
resultpath = os.path.join(data_dir, categories[0], 'train', 'product_training.json')
trainingpath = os.path.join(data_dir, categories[0], 'train', 'review_training.json')
#reading the file with json
reviewdata = pd.read_json(trainingpath)
resultpath = pd.read_json(resultpath)

In [3]:
#combine the 2 shits together
fulldata = resultpath.merge(reviewdata, how='outer', on='asin')

In [11]:
#getting the shit for the decision tree
datatext = fulldata[["verified","image"]]
datatext = datatext.replace(True, 1)
datatext = datatext.replace(False, 0)
datatext = datatext.fillna(0)
for v in datatext["image"]:
    if v != 0:
        v = 1

In [5]:
#DUMP ALL THE SHIT IN FOR NAVIE BAYS
# Load libraries
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

In [9]:
#set the x and y variables
x = datatext
y = fulldata.awesomeness

,verified,image
0,1,0
1,0,0
2,1,0
3,0,0
4,0,0
...,...,...
770781,0,0
770782,0,0
770783,0,0
770784,0,0


In [7]:
#split the datainto different shits
xtrain , xtest, ytrain, ytest = train_test_split(x, y, test_size = .3, random_state = 1)

In [8]:
#trying out the decision tree classif
testC = DecisionTreeClassifier()
testC = testC.fit(xtrain,ytrain)
#y_pred = testC.predict(xtest)

ValueError: setting an array element with a sequence.